In [ ]:
from scipy.stats.mstats import winsorize
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


# 获取指标数据，相当于回测平台的history_bar
get_price("000001.XSHE", start_date="2017-01-01",end_date="2017-01-04")


# 指定获取收盘价数据
get_price(['000024.XSHE', '000001.XSHE', '000002.XSHE'], start_date="2015-04-01",end_date="2015-04-12",fields="close")
# 若不指定指标，则是panel结构


# 获取交易日列表
# 返回datetime.date list - 交易日期列表(除去周末、节假日)
get_trading_dates(start_date="2017-01-01",end_date="2018-01-01")


# 获取财务数据
q = query(fundamentals.income_statement.revenue, 
          fundamentals.income_statement.cost_of_goods_sold
         )
# 必须指定日期
fund = get_fundamentals(q,entry_date="2017-01-03")    # 3维度数据，格式为panel；[itmes, major, minor]对应[指标，时间，股票]

# 面板数据 转 截面数据，panel 转 DataFrame
print(type(fund[:,0,:]))    # DataFrame
fund[:,0,:]    # 列=指标名，行=股票名


## 数据处理：去极值，标准化，市值中性化处理：

In [ ]:
fund = get_fundamentals(query(fundamentals.eod_derivative_indicator.pe_ratio), entry_date="20180102")[:,0,:]

### 1. 去极值

In [ ]:
# 将左右两端2.5%的数据拉回到四分位点的位置，并新增一列【指标】到fund
fund["pe_ratio_winsorize"] = winsorize(fund["pe_ratio"],limits=0.025)


# 画图对比: 取500个数据演示
fund["pe_ratio"][:500].plot()
fund["pe_ratio_winsorize"][:500].plot()


# 分位数去极值，与winsorize功能一样
def percentile(factor,up,down): 
    """自实现分位数去极值，up与down都是percentile"""
    up_scale = np.percentile(factor, up)   
    down_scale = np.percentile(factor, down)   
    factor = np.where(factor > up_scale, up_scale, factor)   
    factor = np.where(factor < down_scale, down_scale, factor)  
    return factor


percentile(fund["pe_ratio"], 97.5, 2.5)


# 中位数去极值
def mad(factor):
    """自实现3倍中位数绝对值偏差去极值"""
    # 1、找出因子的中位数 median
    median_factor = np.median(factor)
    
    # 2、得到每个因子值与中位数的绝对偏差值 |x – median|
    # 3、得到绝对偏差值的中位数， MAD = median(|x – median|)
    mad = np.median(abs(factor-median_factor))
    
    # 4、计算MAD_e = 1.4826*MAD，然后确定参数 n，做出调整
    # 定义3倍中位数的上下限制
    high = median_factor + (3 * 1.4826 * mad)    # 上限
    low = median_factor - (3 * 1.4826 * mad)     # 下限
    
    # 利用3倍中位数的值去极值
    factor = np.where(factor > high, high, factor)    # 如果参数大于极值，则把参数换成极值，否则还是参数
    factor = np.where(factor < low, low, factor)      # 如果参数小于极值，则把参数换成极值，否则还是参数
    return factor


# 对pe-ratio进行3倍中位数去极值
fund["pe_ratio_3mad"] = mad(fund["pe_ratio"])

fund["pe_ratio"][:500].plot()
fund["pe_ratio_3mad"][:500].plot()


# 3sigma去极值
def three_sigma(factor):
    """自定义3倍标准差方法去极值"""
    # 计算平均值和标准差
    mean = factor.mean()
    std_dev = factor.std()
    
    # 计算上下限数据，左右的数据加减3个标准差
    high = mean + (3 * std_dev)
    low = mean - (3 * std_dev)

    # 替换极值数据
    factor = np.where(factor > high, high, factor)
    factor = np.where(factor < low, low, factor)
    return factor


# 对fund["pe_ratio"] 去极值
fund["pe_ratio_3sigma"] = three_sigma(fund["pe_ratio"])

fund['pe_ratio'][:500].plot()
fund['pe_ratio_3sigma'][:500].plot()

## 2. 标准化处理

In [ ]:
std = StandardScaler()
# 先去掉none，再标准化
test1 = std.fit_transform(fund["pe_ratio_3mad"].dropna())    # numpy.ndarray


def stand(factor):
    """自实现标准化，相当于StandScaler"""
    mean = factor.mean()
    std_dev = factor.std()
    return (factor-mean)/std_dev


fund["pe_ratio_stand"] = stand(fund["pe_ratio_3mad"])    # pandas.core.series.Series

## 3. 市值中性化处理

In [ ]:
# 1. 获取数据
q = query(fundamentals.eod_derivative_indicator.pb_ratio,
         fundamentals.eod_derivative_indicator.market_cap)
fund = get_fundamentals(q, entry_date="2018-01-03")[:,0,:]

# 2. 对因子数据进行处理，去极值（3倍中位数），中心化（stand）
fund["pb_ratio"] = stand(mad(fund["pb_ratio"]))
fund["market_cap"] = stand(mad(fund["market_cap"]))    # 可以不用处理数据

# 3. 市值中性化，确定回归方程的特征值（市值因子）和目标值（市净率因子）
# 传入训练的特征值是二维形状
x = fund["market_cap"].reshape(-1,1)    # 通过reshape转化成2维形状；类型：从pandas.core.series.Series变成numpy.ndarray
y = fund["pb_ratio"]    # 1维形状，类型：numpy.ndarray

# 4. 利用线性回归进行预测
lr = LinearRegression()
lr.fit(x,y)

# 5. 得出每个市值因子的预测值，
y_predict = lr.predict(x)
# 市净率因子的真实值 - 预测值（市值因子的y） = 误差，误差就是中心化处理后的结果
fund["pb_ratio"] = y - y_predict

# 取出（行）股票名
# fund["pb_ratio"].index